In [1]:
%%bash
chia version
cdv --version

1.3.5
cdv, version 1.0.8


# Tic Tac Toe Coin

[tic-tac-toe puzzle](tic-tac-toe-ipynb) is an inner puzzle.

[p2_delegated_puzzle_or_hidden_puzzle.py](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py)

In [1]:
from chia.types.blockchain_format.program import Program
from cdv.util.load_clvm import load_clvm
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')

In [2]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet

network: Network = await Network.create()

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

alice: 2000000000000


In [2]:
# empty board
board = [0] * 9
# alice is the 'x'
# and will create an empty board
player = 1
# prepare the tic-tac-toe inner puzzle
inner_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
# alice plays position 4
inner_solution = Program.to([4])

# test tic_tac_toe puzzle
run_result = inner_puzzle.run(inner_solution)
is_winning, board = tic_tac_toe.get_play_result(run_result)
print(is_winning)
tic_tac_toe.print_board(board)


False
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [3]:
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.').curry(tic_tac_toe_puzzle)

# player 2 (o)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, 2)
print(tic_tac_toe_puzzle.get_tree_hash())
print(curried_tic_tac_toe_puzzle.get_tree_hash())

# next_v is 1
solution = Program.to([curried_tic_tac_toe_puzzle, 5, 1])
#print(disassemble(solution))
coin_run_result = tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(coin_run_result))
#print(coin_run_result.get_tree_hash())
run_result = curried_tic_tac_toe_puzzle.run(Program.to([5]))
is_winning, board = tic_tac_toe.get_play_result(run_result)
print(is_winning)
tic_tac_toe.print_board(board)

# player 1 (x)
# next puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, 1)
print(curried_tic_tac_toe_puzzle.get_tree_hash())
#print(disassemble(curried_tic_tac_toe_puzzle))


671ebebd93fb2538976c3483c86ec0cfd759dc10727ad98ad1b6afa5defcdf2b
94fcc7039309f6bba4385b3b483b0f583afa47a1efa456c5f42b700a724fdf6e
0xb7a509c44b99922025f70933f6f9430cd5bf0883ed8006bb6c181105a1abf79a
False
   |   |   
---+---+---
   | x | o 
---+---+---
   |   |   
b7a509c44b99922025f70933f6f9430cd5bf0883ed8006bb6c181105a1abf79a
